### **Libraries needed for NLP**

In [154]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### **Libraries needed for Tensorflow processing**

In [155]:
import tensorflow as tf
import numpy as np
import tflearn
import random
import json

In [156]:
from google.colab import files
files.upload()

Saving intents.json to intents (5).json


{'intents.json': b'{"intents": [\n        {"tag": "greeting",\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day"],\n         "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],\n         "context_set": ""\n        },\n        {"tag": "goodbye",\n         "patterns": ["Bye", "See you later", "Goodbye"],\n         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\n        },\n        {"tag": "thanks",\n         "patterns": ["Thanks", "Thank you", "That\'s helpful"],\n         "responses": ["Happy to help!", "Any time!", "My pleasure"]\n        },\n        {"tag": "hours",\n         "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],\n         "responses": ["We\'re open every day 9am-9pm", "Our hours are 9am-9pm every day"]\n        },\n        {"tag": "location",\n         "patterns": ["What is your location?", "Where are 

### **import our chat-bot intents file**

In [157]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [158]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'hours',
   'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of Lon

In [159]:
words = []
classes = []
documents = []
ignore = ['?']

**loop through each sentence in the intent's patterns**

In [160]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

**Perform stemming and lower each word as well as remove duplicates**

In [161]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

**remove duplicate classes**

In [162]:
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)


31 documents
9 classes ['deliveryoption', 'goodbye', 'greeting', 'hours', 'location', 'menu', 'payments', 'thanks', 'todaysmenu']
57 unique stemmed words ["'s", 'acceiv', 'address', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'cours', 'credit', 'day', 'del', 'delicy', 'delivery', 'dish', 'do', 'food', 'for', 'from', 'good', 'goodby', 'hello', 'help', 'hi', 'hom', 'hour', 'how', 'is', 'lat', 'loc', 'main', 'mastercard', 'me', 'menu', 'most', 'on', 'op', 'opt', 'provid', 'resta', 'see', 'serv', 'situ', 'spec', 'tak', 'tel', 'thank', 'that', 'the', 'ther', 'today', 'what', 'when', 'wher', 'yo', 'you']


**create training data**

In [163]:
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

**create training set, bag of words for each sentence**

In [164]:
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

**output is '1' for current tag and '0' for rest of other tags**

In [165]:
 output_row = list(output_empty)
 output_row[classes.index(doc[1])] = 1

 training.append([bag, output_row])


**shuffling features and turning it into np.array**

In [166]:
random.shuffle(training)
training = np.array(training)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


 creating training lists

In [167]:
train_x = list(training[:,0])
train_y = list(training[:,1])

**resetting underlying graph data**

In [168]:
tf.compat.v1.reset_default_graph()

**Building neural network**

In [169]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)



** Defining model and setting up tensorboard**

In [170]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

**Start training**

In [171]:
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

---------------------------------
Run id: 6FEHP7
Log directory: tflearn_logs/
---------------------------------
Training samples: 1
Validation samples: 0
--
Training Step: 1  | time: 0.101s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 -- iter: 1/1
--
Training Step: 2  | total loss: 2.15557 | time: 0.004s
| Adam | epoch: 002 | loss: 2.15557 - acc: 0.0000 -- iter: 1/1
--
Training Step: 3  | total loss: 2.18368 | time: 0.004s
| Adam | epoch: 003 | loss: 2.18368 - acc: 0.8182 -- iter: 1/1
--
Training Step: 4  | total loss: 2.18368 | time: 0.003s
| Adam | epoch: 004 | loss: 2.18368 - acc: 0.9545 -- iter: 1/1
--
Training Step: 5  | total loss: 2.18876 | time: 0.006s
| Adam | epoch: 005 | loss: 2.18876 - acc: 0.9860 -- iter: 1/1
--
Training Step: 6  | total loss: 2.18876 | time: 0.007s
| Adam | epoch: 006 | loss: 2.18876 - acc: 0.9950 -- iter: 1/1
--
Training Step: 7  | total loss: 2.18566 | time: 0.005s
| Adam | epoch: 007 | loss: 2.18566 - acc: 0.9980 -- iter: 1/1
--
Training Step: 8  

In [172]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

**restoring all the data structures**

In [173]:
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


In [174]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

**load the saved model**

In [175]:
model.load('./model.tflearn')

In [176]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

**returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence**

In [177]:
def bow(sentence, words, show_details=False):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))


In [178]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

In [179]:
def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [180]:
classify('What are your hours of operation?')

response('What are your hours of operation?')

response('What is menu for today?')


You can visit www.mymenu.com for menu options
You can visit www.mymenu.com for menu options


In [ ]:
response('Do you accept Credit Card?')